Este projeto consiste no desenvolvimento de um sistema de assistência virtual utilizando técnicas de Processamento de Linguagem Natural (PLN).
O sistema deve ser capaz de converter texto em fala e fala em texto, além de interpretar comandos de voz para executar ações automatizadas.


(English Translate)


The project consists of developing a virtual assistant system using Natural Language Processing (NLP) techniques.  
The system must be able to convert text into speech and speech into text, as well as interpret voice commands to perform automated actions.

O primeiro passo é importar as bibliotecas que serão utilizadas ao longo do código

(English Translate)

The first step is to import the libraries that will be used throughout the code.

In [ ]:
import base64
import speech_recognition as sr
from gtts import gTTS
import IPython.display as ipd
from IPython.display import display, Javascript
from google.colab import output
import wikipediaapi
from pydub import AudioSegment
import io

Configuração para que a API do Wikipedia funcione corretamente

(English Translate)

Setup for the Wikipedia API to function properly

In [ ]:
#---------------------------------
# CONFIGURAÇÃO
# --------------------------------

wiki_wiki = wikipediaapi.Wikipedia('AssistenteVirtual/1.0 (contato@exemplo.com)', 'pt')


Criação de funções

(English Translate)

Defining functions

In [ ]:

#---------------------------------
# Módulo 1: Text to Speech
# --------------------------------

def speak(text):
    print(f"Assistente: {text}")
    tts = gTTS(text=text, lang='pt')
    tts.save("res.mp3")
    display(ipd.Audio("res.mp3", autoplay=True))


#---------------------------------
#Captura áudio do navegador e converte para WAV PCM
# --------------------------------

def record_and_convert_audio(seconds=5):
    js = f"""
    async function record() {{
        const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
        const recorder = new MediaRecorder(stream);
        let chunks = [];
        recorder.ondataavailable = e => chunks.push(e.data);
        recorder.start();
        await new Promise(r => setTimeout(r, {seconds * 1000}));
        recorder.stop();
        return new Promise(r => {{
            recorder.onstop = async () => {{
                const blob = new Blob(chunks);
                const reader = new FileReader();
                reader.readAsDataURL(blob);
                reader.onloadend = () => r(reader.result);
            }};
        }});
    }}
    """
    display(Javascript(js))
    data = output.eval_js('record()')

    # Decodifica o base64 (formato webm/ogg do navegador)
    audio_bytes = base64.b64decode(data.split(',')[1])

    # Converte WebM/OGG para WAV usando pydub
    audio_segment = AudioSegment.from_file(io.BytesIO(audio_bytes))
    audio_segment.export("audio_final.wav", format="wav")
    return "audio_final.wav"


#---------------------------------
# Módulo 2: Speech to Text
# --------------------------------

def listen():
    recognizer = sr.Recognizer()
    try:
        audio_path = record_and_convert_audio()
        with sr.AudioFile(audio_path) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data, language='pt-BR')
            print(f"Você disse: {text}")
            return text.lower()
    except Exception as e:
        print(f"Erro no processamento: {e}")
        return ""

#---------------------------------
# Módulo 3: Coordena toda a interação da assistente
# --------------------------------

def run_assistant():
    speak("Estou ouvindo. Diga seu comando.")
    #speak('Estou ouvindo. Diga "faculdade", "igreja", "youtube" ou "farmácia"')
    command = listen()

    if 'faculdade' in command:
        speak("Abrindo o site da faculdade Uninter, você precisa estudar.")
        display(Javascript('window.open("https://univirtus.uninter.com/ava/web/#/ava", "_blank");'))

    elif 'igreja' in command:
        speak("Abrindo o site da Asembleia de Deus do Brás.")
        display(Javascript('window.open("https://adbras.com.br/sobre-a-igreja/", "_blank");'))


    elif 'youtube' in command:
        speak("É hora de relaxar, abrindo o site do Youtube mas só pode assistir vídeos em inglês!")
        display(Javascript('window.open("https://www.youtube.com", "_blank");'))

    elif 'farmácia' in command:
        speak("Buscando farmácias no Google Maps, de tanto estudar já está com dor de cabeça.")
        url = "https://www.google.com"
        display(Javascript(f'window.open("{url}", "_blank");'))
        print("Dica: Use o [Google Maps](https://www.google.com) para ver telefone e endereço.")

    else:
        speak("Comando não reconhecido ou áudio muito curto.")

# Chamada da função
run_assistant()


Assistente: Estou ouvindo. Diga seu comando.


<IPython.core.display.Javascript object>

Você disse: faculdade
Assistente: Abrindo o site da faculdade Uninter, você precisa estudar.


<IPython.core.display.Javascript object>